In [1]:
from enum import IntEnum
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import trange
from collections import defaultdict

from GrU import GrU

### Load auction data

In [2]:
class clock( IntEnum ):
    pv = 0
    storage = 1
    charge = 2
    discharge = 3

class source( IntEnum ):
    grid =  0
    pv = 1
    storage = 2
    charge = 3
    discharge = 4

df_auctioneer_data = pd.read_csv('../data/Auctioneer Data.csv')

headers = np.load(file='../data/clock_data_columns.npy', allow_pickle=True)
df_clock_data = pd.DataFrame(np.load('../data/clock_data_values.npy', allow_pickle=True), columns=headers)

def getPriceVectors(round):
    '''
    Get price vectors for a round from the clock data.

    Parameters
    ----------
    round : int
        Round number to get the price vectors for, range = [0, 308]

    Returns
    -------
    prices : ndarray of shape (5, 24)
        Price vectors for g, p, s, c, d.
    
    '''
    prices = np.ndarray((5, 24))
    prices[0] = df_auctioneer_data.GRID_PRICE.values
    for e in clock:
        prices[e + 1] = df_clock_data[f'price_{round}'][e]
    return prices

## Gamma Determination

In [3]:
ei = [0.00044688, 0.00044796, 0.00045042, 0.00045573, 0.00046029, 0.00046002, 0.00045153, 0.00043737, 0.00042772, 0.00041107, 0.00040774, 0.00038668, 0.00037268, 0.00036565, 0.00036387, 0.00035763, 0.00034319, 0.00034377, 0.00035285, 0.00037902, 0.00039631, 0.00040366, 0.00039713, 0.00039088]
ei_t = np.array(ei) * 1e3

d = np.ones(24) * 5

gamma_true = 10

prices = np.ones(shape=(5, 24))
prices[source.pv] = prices[source.grid] = 1
prices[source.charge] = prices[source.discharge] = prices[source.storage] = np.inf

pi_g_true = prices[source.grid].copy()
prices[source.grid] += gamma_true * ei_t

In [4]:
prices[:2]

array([[5.4688, 5.4796, 5.5042, 5.5573, 5.6029, 5.6002, 5.5153, 5.3737,
        5.2772, 5.1107, 5.0774, 4.8668, 4.7268, 4.6565, 4.6387, 4.5763,
        4.4319, 4.4377, 4.5285, 4.7902, 4.9631, 5.0366, 4.9713, 4.9088],
       [1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
        1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
        1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ]])

In [5]:
r = 0
ep = 1e-3

d_star, _ = GrU(d, prices)
while(not any(d_star[source.pv] == 0)):
    r += 1
    delta = np.array([(24 * (r-1) + t) for t in range(24)])
    prices[source.pv] = pi_g_true + ep * delta * ei_t

    d_star, _ = GrU(d, prices)

t = np.argwhere(d_star[source.pv] == 0)[0].item()
gamma = (24 * (r-1) + t) * ep
print(r, t, gamma)

/home/naman/Desktop/Thesis/DER-auction/main/GrU.py:86: RuntimeWarning: invalid value encountered in subtract
  exportPrice = cumStoragePrice[ t-1 ] - np.append( 0, cumStoragePrice[ 0: t-1 ] )


417 17 10.001
